## CS3570 Introduction to Multimedia Technology
### Homework 02 Due: 11:59pm, 4/11/2025

#### In the first part:
You are required to implement DCT compression with different quality factors with the given image file (cat.jpg), and calculate the SSIM score of between the original image and compressed images.

Here's the step:
1. Convert the image from RGB to YCbCr following with chrominance subsampling 4:2:0.
2. Divide the Y, Cb, Cr channels into blocks of 8 x 8 pixels.
3. Shift values by -128 and transform the each 8x8 blocks from spatial domain to the DCT domain.
4. Apply quantization with the provided quantization tables and quality factor.
5. Reconstruct the image by taking inverse discrete cosine transform (IDCT) and shift values by +128.
6. Reassemble the Y, Cb, Cr channels from 8x8 blocks.
7. Upsample Cb, Cr and convert YCbCr back to RGB.
8. Compute the PSNR score with reconstructed images and the original image.

#### In the second part:
You need to design and apply different FIR filters into separate three audio signals from the given audio file (HW2_Mix.wav). Next, you are asked to reduce the sampling rate of filtered signals. Finally, since the output audio signals are too simple, you should apply one-fold echo and multiple-fold echo to produce more complex music.

Here's the step:
1. Transform the input signal into frequency domain and plot the spectrum (magnitude in spectrum should be normalized to [0, 1]).
2. Implement 3 different FIR filters to separate the three audio signals with Blackman window function (You have to pick the appropriate filter size, cut-off frequency, and window size).
3. Implement 1D convolution on the input signal with your filters (zero padding).
4. Reduce the sampling rates of the three separated songs to 2000Hz.
5. Apply one-fold echo and multiple-fold echo on the signal that pass through the **low-pass filter**. (Please use the audio files before reducing sampling rates)

### Reminder:
* **The imported libraries are sufficient for this homework, you are not permitted to import other Python packages.**
* **Your code must display and output your results to enable us to verify its correctness.**
* **Please follow the instructions in the Jupyter Notebook and complete the parts marked as `"TODO."`**
* **Please compress your Jupyter Notebook file, image results from Q1 and audio results from Q2 in a zip file named HW2_xxxxxxxxx_ooo.zip, where xxxxxxxxx is your student ID and ooo is your name.**
* **Homework should be submitted before the announced due time. Scores of late homework will be reduced by 20% per day.**
* **If you encounter any problems or have questions, please post them on eeclass.**

## Part 1: DCT compression (40%)

### Import the necessary libraries

In [1]:
# !pip install opencv-python numpy matplotlib scipy

import numpy as np
from cv2 import imread, imwrite
import matplotlib.pyplot as plt

### Define the quantization tables

In [2]:
# Quantization table for Y
QT_L = np.array([
    [16, 11, 10, 16, 24, 40, 51, 61],
    [12, 12, 14, 19, 26, 58, 60, 55],
    [14, 13, 16, 24, 40, 57, 69, 56],
    [14, 17, 22, 29, 51, 87, 80, 62],
    [18, 22, 37, 56, 68, 109, 103, 77],
    [24, 36, 55, 64, 81, 104, 113, 92],
    [49, 64, 78, 87, 103, 121, 120, 101],
    [72, 92, 95, 98, 112, 100, 103, 99]
])

# Quantization table for Cb, Cr
QT_Chroma = np.array([
    [17, 18, 24, 47, 99, 99, 99, 99],
    [18, 21, 26, 66, 99, 99, 99, 99],
    [24, 26, 56, 99, 99, 99, 99, 99],
    [47, 66, 99, 99, 99, 99, 99, 99],
    [99, 99, 99, 99, 99, 99, 99, 99],
    [99, 99, 99, 99, 99, 99, 99, 99],
    [99, 99, 99, 99, 99, 99, 99, 99],
    [99, 99, 99, 99, 99, 99, 99, 99]
])

### 1. Implement RGB to YCbCr and chrominance subsampling (4:2:0) (5%)

In [3]:
def RGB2YCbCr(img):
    '''
    Input:
        img: input image (RGB)
    Output:
        Y: Y channel
        Cb: Cb channel
        Cr: Cr channel
    '''
    H, W, _ = img.shape
    img_ycbcr = np.zeros(img.shape)
    Y = np.zeros((H, W))
    Cb = np.zeros((H//2, W//2))
    Cr = np.zeros((H//2, W//2))

    # matrices for calculating RGB to YCbCr
    M = np.array([[0.257, 0.564, 0.098],
                  [-0.148, -0.291, 0.439],
                  [0.439, -0.368, -0.071]])
    C = np.array([16, 128, 128])

    # [TODO]: Convert RGB to YCbCr with chrominance subsampling (4:2:0)
    for i in range(H):
        for j in range(W):
            rgb = img[i, j].astype(float)
            ycbcr = np.dot(M, rgb) + C
            Y[i, j] = ycbcr[0]
            img_ycbcr[i, j] = ycbcr
    for i in range(0, H, 2):
        for j in range(0, W, 2):
            max_i = min(i + 2, H)
            max_j = min(j + 2, W)
            
            cb_sum = 0
            cr_sum = 0
            count = 0
            for bi in range(i, max_i):
                for bj in range(j, max_j):
                    cb_sum += img_ycbcr[bi, bj, 1]
                    cr_sum += img_ycbcr[bi, bj, 2]
                    count += 1
            Cb[i//2, j//2] = cb_sum / count
            Cr[i//2, j//2] = cr_sum / count
    return Y, Cb, Cr

### 2. Implement YCbCr to RGB (5%)

In [5]:
def YCbCr2RGB(Y, Cb, Cr):
    '''
    Input:
        Y: Y channel
        Cb: Cb channel
        Cr: Cr channel
    Output:
        img: output image (RGB)
    '''
    H, W = Y.shape
    # [TODO]: Upsample Cb and Cr
    Cb_upsample = np.repeat(np.repeat(Cb, 2, axis=0), 2, axis=1)[:H, :W]
    Cr_upsample = np.repeat(np.repeat(Cr, 2, axis=0), 2, axis=1)[:H, :W]
    img_YCbCr = np.stack((Y, Cb_upsample, Cr_upsample), axis=-1)
    img = np.zeros((H, W, 3))

    # matrices for converting YCbCr to RGB
    M = np.array([[1.164, 0, 1.596],
                  [1.164, -0.392, -0.813],
                  [1.164, 2.017, 0]])
    C = np.array([16, 128, 128])

    # [TODO]: Convert YCbCr back to RGB
    for i in range(H):
            for j in range(W):
                ycbcr = img_YCbCr[i, j].astype(float)
                rgb = np.dot(M, (ycbcr - C))
                img[i, j] = np.clip(rgb, 0, 255)
    return img

### 3. Implement Discrete Cosine Transform (DCT) (5%)

In [6]:
def DCT(f):
    '''
    Input:
        f: image block
    Output:
        F: DCT transformed image block
    '''
    # [TODO]: Implement discrete cosine transform
    # Assuming f is an 8x8 block
    N = 8
    F = np.zeros((N, N))
    
    # DCT transformation
    for u in range(N):
        for v in range(N):
            # Normalization factors
            cu = 1.0 / np.sqrt(2) if u == 0 else 1.0
            cv = 1.0 / np.sqrt(2) if v == 0 else 1.0
            
            # Calculate the DCT coefficient at (u,v)
            sum_val = 0.0
            for x in range(N):
                for y in range(N):
                    # Apply the DCT formula
                    sum_val += f[x, y] * np.cos((2*x+1)*u*np.pi/(2*N)) * np.cos((2*y+1)*v*np.pi/(2*N))
            
            # Apply scaling factors
            F[u, v] = (2.0 / N) * cu * cv * sum_val
    return F

### 4. Implement Inverse Discrete Cosine Transform (IDCT) (5%)

In [7]:
def IDCT(F):
    '''
    Input:
        F: DCT transformed image block
    Output:
        f: image block
    '''
    # [TODO]: Implement inverse discrete cosine transform
    # Assuming F is an 8x8 block
    N = 8
    f = np.zeros((N, N))
    
    # IDCT transformation
    for x in range(N):
        for y in range(N):
            # Calculate the pixel value at (x,y)
            sum_val = 0.0
            for u in range(N):
                for v in range(N):
                    # Normalization factors
                    cu = 1.0 / np.sqrt(2) if u == 0 else 1.0
                    cv = 1.0 / np.sqrt(2) if v == 0 else 1.0
                    
                    # Apply the IDCT formula
                    sum_val += cu * cv * F[u, v] * np.cos((2*x+1)*u*np.pi/(2*N)) * np.cos((2*y+1)*v*np.pi/(2*N))
            
            # Apply scaling factor
            f[x, y] = (2.0 / N) * sum_val
    return f

### 5. Implement quantization with quality factor (5%)

In [8]:
def quantize(img, Qf, chroma=True):
    '''
    Input:
        img: image block
        Qf: quality factor
    Output:
        img: quantized image block
    '''
    # [TODO]: Implement quantization
    # Calculate scaling factor based on quality factor
    if Qf >= 50:
        S = (100 - Qf) / 50
    else:
        S = 50 / Qf
    
    # Scale the appropriate quantization table
    if chroma:      # if input block is Cb or Cr
        T_s = np.floor(QT_Chroma * S + 0.5)
    else:           # if input block is Y
        T_s = np.floor(QT_L * S + 0.5)
    
    # Prevent division by zero
    T_s = np.maximum(T_s, 1)
    
    # Apply quantization by dividing and rounding
    img = np.round(img / T_s)
    return img

### 6. DCT compression process (10%)

In [ ]:
def dct_compression(img, Qf):
    '''
    Input:
        img: input image
        Qf: quality factor
    Output:
        compressed: compressed image
    '''
    height, weight, _ = img.shape
    block_size = 8

    Y, Cb, Cr = RGB2YCbCr(img)
    Y_compressed = np.zeros(Y.shape)
    Cb_compressed = np.zeros(Cb.shape)
    Cr_compressed = np.zeros(Cr.shape)

    # [TODO]: Implement the DCT compression process

    # Process Y channel (luminance)
    for i in range(0, height, block_size):
        for j in range(0, weight, block_size):
            # Get block boundaries (handle edge cases)
            i_end = min(i + block_size, height)
            j_end = min(j + block_size, weight)
            
            # Extract block and pad if needed
            block = np.zeros((block_size, block_size))
            block[:i_end-i, :j_end-j] = Y[i:i_end, j:j_end]
            
            # Step 3: Shift values by -128 and apply DCT
            dct_block = DCT(block - 128)
            
            # Step 4: Apply quantization
            quantized = quantize(dct_block, Qf, chroma=False)
            
            # Step 5: Inverse DCT and shift by +128
            reconstructed = IDCT(quantized) + 128
            
            # Store in output array
            Y_compressed[i:i_end, j:j_end] = reconstructed[:i_end-i, :j_end-j]

    # Process Cb, Cr channels (chrominance) - already subsampled
    cb_height, cb_width = Cb.shape
    for i in range(0, cb_height, block_size):
        for j in range(0, cb_width, block_size):
            # Get block boundaries (handle edge cases)
            i_end = min(i + block_size, cb_height)
            j_end = min(j + block_size, cb_width)
            
            # Process Cb channel
            cb_block = np.zeros((block_size, block_size))
            cb_block[:i_end-i, :j_end-j] = Cb[i:i_end, j:j_end]
            
            dct_cb = DCT(cb_block - 128)
            quantized_cb = quantize(dct_cb, Qf, chroma=True)
            reconstructed_cb = IDCT(quantized_cb) + 128
            
            Cb_compressed[i:i_end, j:j_end] = reconstructed_cb[:i_end-i, :j_end-j]
            
            # Process Cr channel
            cr_block = np.zeros((block_size, block_size))
            cr_block[:i_end-i, :j_end-j] = Cr[i:i_end, j:j_end]
            
            dct_cr = DCT(cr_block - 128)
            quantized_cr = quantize(dct_cr, Qf, chroma=True)
            reconstructed_cr = IDCT(quantized_cr) + 128
            
            Cr_compressed[i:i_end, j:j_end] = reconstructed_cr[:i_end-i, :j_end-j]
    
    # Step 7: Convert YCbCr back to RGB
    img_compressed = YCbCr2RGB(Y_compressed, Cb_compressed, Cr_compressed)

    return np.clip(img_compressed, 0, 255).astype(np.uint8)

### 7. Implement Peak Signal-to-Noise Ratio (PSNR) (5%)

In [10]:
def PSNR(original, compressed):
    '''
    Input:
        original: original image
        compressed: compressed image
    Output:
        psnr: PSNR value
    '''
    # [TODO]: Calculate PSNR from the original image and the compressed image
    # Convert images to float for calculation accuracy
    original = original.astype(np.float64)
    compressed = compressed.astype(np.float64)
    
    # Calculate Mean Squared Error (MSE)
    mse = np.mean((original - compressed) ** 2)
    
    # Handle the case where images are identical
    if mse == 0:
        return float('inf')
    
    # Maximum possible pixel value for 8-bit image
    max_pixel = 255.0
    
    # Calculate PSNR
    psnr = 20 * np.log10(max_pixel / np.sqrt(mse))
    return psnr

### Experiment with different quality factor

In [11]:
image = imread('Barbara.jpg')

Qf = 80
compressed_img = dct_compression(image, Qf)
print(f"PSNR (Qf={Qf}): {PSNR(image, compressed_img)}")
imwrite(f'Barbara_quality-{Qf}.jpg', compressed_img)

Qf = 30
compressed_img = dct_compression(image, Qf)
print(f"PSNR (Qf={Qf}): {PSNR(image, compressed_img)}")
imwrite(f'Barbara_quality-{Qf}.jpg', compressed_img)

Qf = 5
compressed_img = dct_compression(image, Qf)
print(f"PSNR (Qf={Qf}): {PSNR(image, compressed_img)}")
imwrite(f'Barbara_quality-{Qf}.jpg', compressed_img)

NameError: name 'width' is not defined

## Part 2: FIR Filter (40%)

In [ ]:
from scipy.io import wavfile
from scipy.fft import fft, fftfreq

### 1. Plot the frequency spectrum (5%)

In [ ]:
def plot_spectrum(data, title, fs, xbond, ybond):
    '''
    Input:
        data: input signal
        title: title of the plot
        fs: sampling frequency
        xbond: x-axis range
        ybond: y-axis range
    '''
    # TODO: FFT & Plot the magnitude spectrum of the input signal

    plt.figure()
    plt.xlim(xbond[0], xbond[1])
    plt.ylim(ybond[0], ybond[1])
    plt.plot(result[:, 0], result[:, 1])
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Magnitude')
    plt.title(title)

### 2. Implement three different filters - Lowpass, Highpass, Bandpass (10%)

In [ ]:
def filter_generator(sample_rate, cutoff, filter_type, window_type, N):
    '''
    Input:
        sample_rate: sampling frequency
        cutoff: cutoff frequency
        filter_type: type of filter (Lowpass, Highpass, Bandpass)
        window_type: type of window function (Blackmann)
        N: filter length (number of points in a filter), also the window length
    Output:
        filter: low-pass, high-pass, or band-pass filter
    '''
    # Normalize cutoff frequency
    if filter_type == 'Lowpass' or filter_type == 'Highpass':
        cutoff = cutoff / sample_rate
    elif filter_type == 'Bandpass':
        cutoff = [cutoff[0] / sample_rate, cutoff[1] / sample_rate]
    else:
        raise ValueError('Invalid filter type')

    mid = N // 2
    n = np.arange(-mid, mid+1)
    n[mid] = 1
    filter = np.zeros(2 * mid + 1)

    # implement the ideal filter
    if filter_type == 'Lowpass':
        # [TODO]: Implement the Low-pass filter
        pass
    elif filter_type == 'Highpass':
        # [TODO]: Implement the High-pass filter
        pass
    elif filter_type == 'Bandpass':
        # [TODO]: Implement the Band-pass filter
        pass

    n = np.arange(0, N)
    # implement the window function
    if window_type == "Blackmann":
        # [TODO]: Implement Blackmann window function
        pass

    return filter

In [ ]:
def plot_filter(filter, title):

    plt.figure()
    plt.plot(filter)
    plt.title(title)
    plt.xlabel('samples')
    plt.ylabel('amplitude')
    plt.show()

### 3. Implement convolution function (10%)

In [ ]:
def convolve(signal , filter , N):
    '''
    Input:
        signal: input signal
        filter: filter
        N: filter length
    Output:
        out: output signal
    '''
    len_signal = len(signal)
    out = np.zeros(len(signal))

    # [TODO]: Implement the 1D convolution operation

    return out

### 4. Reduce sample rate (5%)

In [ ]:
def down_sampling(signal, fs, new_fs):
    '''
    Input:
        signal: input signal
        fs: sampling frequency
        new_fs: new sampling frequency
    Output:
        output_signal: down-sampled signal
    '''
    # [TODO]: Implement the down-sampling operation

    return signal

### 5. Implement one-fold / multiple-fold echo (10%)

In [ ]:
def echo(signal_low, delay, alpha, fs):
    '''
    Input:
        signal_low: input signal (time-domain)
        delay: delay time
        alpha: attenuation factor
        fs: sampling frequency
    Output:
        output_echo_one: output signal with one-fold echo
        output_echo_multiple: output signal with multiple-fold echo
    '''

    D = int(delay * fs)
    input_len = len(signal_low)
    output_echo_one = np.zeros(input_len + D)

    #[TODO]: One-fold echo


    D = int(delay * fs)
    input_len = len(signal_low)
    output_echo_multiple = np.zeros(input_len + D)

    #[TODO]: Multiple-fold echo


    output_echo_one = output_echo_one.astype(np.float32)
    output_echo_multiple = output_echo_multiple.astype(np.float32)

    return  output_echo_one, output_echo_multiple

### Note: There should be 10 images displayed below the cell.

In [ ]:
# Read the wav file
fs, data = wavfile.read('HW2_Mix.wav')

# Plot the magnitude spectrum of the input signal
plot_spectrum(data, "Original spectrum", fs, [0, 1500], [0, 2])

# [TODO]: determine the cutoff frequency and window size
fc_low =
fc_high =
fc_band = np.array([XXX, xxx])
filter_size =

# Generate filter with Blackmann window - Lowpass, Highpass, Bandpass
lowpass_blackmann_filter = filter_generator(fs, fc_low, 'Lowpass', 'Blackmann', filter_size)
highpass_blackmann_filter = filter_generator(fs, fc_high, 'Highpass', 'Blackmann', filter_size)
bandpass_blackmann_filter = filter_generator(fs, fc_band, 'Bandpass', 'Blackmann', filter_size)

# Plot filter shape
plot_filter(lowpass_blackmann_filter, "Lowpass Filter - Blackmann Window")
plot_filter(highpass_blackmann_filter, "Highpass Filter - Blackmann Window")
plot_filter(bandpass_blackmann_filter, "Bandpass Filter - Blackmann Window")

# Plot the magnitude spectrum of different filters
plot_spectrum(lowpass_blackmann_filter, "Lowpass Filter Spectrum - Blackmann Window", fs, [0, 1500], [0, 2])
plot_spectrum(highpass_blackmann_filter, "Highpass Filter Spectrum - Blackmann Window", fs, [0, 1500], [0, 2])
plot_spectrum(bandpass_blackmann_filter, "Bandpass Filter Spectrum - Blackmann Window", fs, [0, 1500], [0, 2])

# Convolve the input signal with different filters
signal_low = convolve(data, lowpass_blackmann_filter, filter_size)
signal_high = convolve(data, highpass_blackmann_filter, filter_size)
signal_band = convolve(data, bandpass_blackmann_filter, filter_size)
plot_spectrum(signal_low, "Lowpass signal - Blackmann Window", fs, [0, 1500], [0, 2])
plot_spectrum(signal_high, "Highpass signal - Blackmann Window", fs, [0, 1500], [0, 2])
plot_spectrum(signal_band, "Bandpass signal - Blackmann Window", fs, [0, 1500], [0, 2])

# save audio
wavfile.write(f"Low_pass_{fc_low}.wav", fs, signal_low)
wavfile.write(f"High_pass_{fc_high}.wav", fs, signal_high)
wavfile.write(f"Band_pass_{fc_band[0]}_{fc_band[1]}.wav", fs, signal_band)

# Reduce sample rate
new_fs = 2000
signal_low_ds = down_sampling(signal_low, fs, new_fs)
signal_high_ds = down_sampling(signal_high, fs, new_fs)
signal_band_ds = down_sampling(signal_band, fs, new_fs)

# You can also try setting 'new_fs' as 'fs' to see what happens.
wavfile.write(f"Low_pass_{fc_low}_2khz.wav", new_fs, signal_low_ds)
wavfile.write(f"High_pass_{fc_high}_2khz.wav", new_fs, signal_high_ds)
wavfile.write(f"Band_pass_{fc_band[0]}_{fc_band[1]}_2khz.wav", new_fs, signal_band_ds)

# Generate echo
echo_one , echo_multiple = echo(signal_low, 1, 0.5, fs)
wavfile.write('Echo_one.wav', fs, echo_one)
wavfile.write("Echo_multiple.wav", fs, echo_multiple)

## Report (20%)

Your report should cover the following aspects:
* DCT compression
    * Describe how you implemented the discrete cosine transform. (4%)
    * Why should we convert image from RGB to YCbCr before compressing? (3%)
    * Explain how can the quality factor affects the compression result. (3%)
* FIR Filter
    * Describe how you implemented the filter and convolutions to separate tha mixed song. And how did you determine the filter size and cut-off frequency. (4%)
    * Compare the spectrum and shape of the filters. (3%)
    * Compare the difference between the signals before and after reducing the sampling rates. (3%)
